In [6]:
from rtree import index
import numpy as np

class RTree3D:
    def __init__(self, triangles):
        p = index.Property(dimension=3)
        self.triangles = triangles
        self.idx = index.Index(interleaved=True, properties=p)
        for i, triangle in enumerate(triangles):
            self.idx.insert(i, self._get_triangle_bounds(triangle), triangle)

    def _get_triangle_bounds(self, triangle):
        min_x = min(triangle.a[0], triangle.b[0], triangle.c[0])
        min_y = min(triangle.a[1], triangle.b[1], triangle.c[1])
        min_z = min(triangle.a[2], triangle.b[2], triangle.c[2])

        max_x = max(triangle.a[0], triangle.b[0], triangle.c[0])
        max_y = max(triangle.a[1], triangle.b[1], triangle.c[1])
        max_z = max(triangle.a[2], triangle.b[2], triangle.c[2])

        return (min_x, min_y, min_z, max_x, max_y, max_z)

    def queryRay(self, origin, direction):
        rayBounds = self._get_ray_bounds(origin, direction)
        print(f"{rayBounds=}")
        if(len(rayBounds) == 0):
            return []
        intersections = self.idx.intersection(rayBounds, objects=True)
        intersected_triangles = [self.triangles[obj.id] for obj in intersections]

        return intersected_triangles

    def _get_ray_bounds(self, origin, direction):
        
        t_min, t_max = 0, 1e9  # Change t_max if you want a maximum ray length

        # Compute the ray intersection with the scene's AABB
        bounds_min = np.array(self.idx.get_bounds()[:3])
        bounds_max = np.array(self.idx.get_bounds()[3:])
        t1 = (bounds_min - origin) / direction
        t2 = (bounds_max - origin) / direction

        t_near = np.maximum(np.minimum(t1, t2), t_min)
        t_far = np.minimum(np.maximum(t1, t2), t_max)

        if np.any(t_near > t_far):
            return []

        bounds_min = origin + direction * t_near
        bounds_max = origin + direction * t_far

        return (*bounds_min, *bounds_max)


In [13]:
import heapq
from rtree import index
import numpy as np

class RTree3D:
    def __init__(self, triangles):
        p = index.Property(dimension=3)
        self.triangles = triangles
        self.idx = index.Index(interleaved=True, properties=p).

        for i, triangle in enumerate(triangles):
            bounds = self._get_triangle_bounds(triangle)
            print(bounds)
            self.idx.insert(i, bounds)

    def _get_triangle_bounds(self, triangle):
        min_x = min(triangle.a[0], triangle.b[0], triangle.c[0])
        min_y = min(triangle.a[1], triangle.b[1], triangle.c[1])
        min_z = min(triangle.a[2], triangle.b[2], triangle.c[2])

        max_x = max(triangle.a[0], triangle.b[0], triangle.c[0])
        max_y = max(triangle.a[1], triangle.b[1], triangle.c[1])
        max_z = max(triangle.a[2], triangle.b[2], triangle.c[2])

        return (min_x, min_y, min_z, max_x, max_y, max_z)

    def queryRay(self, origin, direction):
        intersected_triangles = []
        ray_bbox_visitor = RayBBoxVisitor(self.idx, origin, direction)

        for node_id in ray_bbox_visitor.visit():
            if node_id in self.idx.leaves():
                intersected_triangles.append(self.triangles[node_id])

        return intersected_triangles


class RayBBoxVisitor:
    def __init__(self, rtree, origin, direction):
        self.rtree = rtree
        self.origin = origin
        self.direction = direction
        self.queue = []

    def visit(self):
        root = self.rtree.get_root()
        heapq.heappush(self.queue, (0.0, root))

        while self.queue:
            _, node = heapq.heappop(self.queue)

            if self.rtree.is_leaf(node):
                yield node.id
            else:
                for child in self.rtree.children(node):
                    t_near, t_far = self._ray_bbox_intersection(child.bbox)
                    if t_near <= t_far:
                        heapq.heappush(self.queue, (t_near, child))

    def _ray_bbox_intersection(self, bbox):
        t1 = (bbox[:3] - self.origin) / self.direction
        t2 = (bbox[3:] - self.origin) / self.direction
        t_near = np.maximum(np.minimum(t1, t2), 0)
        t_far = np.minimum(np.maximum(t1, t2), 1e9)

        return float(np.max(t_near)), float(np.min(t_far))


In [1]:

import numpy as np
from rtree import index

from twobounce2 import ObjLoader
from GeometricObjects import *


class Triangle:
    def __init__(self, coords):
        self.a = coords[0]
        self.b = coords[1]
        self.c = coords[2]
        self.at = None
        self.bt = None
        self.ct = None
        self.normal = None
        self.collisions = []
        self.textureCoords = []
        self.bounds = self.bounds()

    def bounds(self):
        min_coords = np.minimum(np.minimum(self.a, self.b), self.c)
        max_coords = np.maximum(np.maximum(self.a, self.b), self.c)
        return tuple(min_coords) + tuple(max_coords)


class RTree3D:
    def __init__(self, triangles):
        self.triangles = triangles
        p = index.Property()
        p.dimension = 3
        self.index = index.Index(properties=p, interleave=True)

        for i, triangle in enumerate(triangles):
            self.index.insert(i, triangle.bounds)

    def queryRay(self, origin, direction):
        ray_origin = np.array(origin)
        ray_direction = np.array(direction) / np.linalg.norm(direction)

        def ray_box_intersection(box_min, box_max):
            t_min = (box_min - ray_origin) / ray_direction
            t_max = (box_max - ray_origin) / ray_direction
            t_min, t_max = np.minimum(t_min, t_max), np.maximum(t_min, t_max)
            t_enter = np.max(t_min)
            t_exit = np.min(t_max)
            return t_enter <= t_exit and t_exit >= 0

        intersecting_triangles = []

        for i in self.index.intersection(origin + direction, objects=True):
            triangle = self.triangles[i.id]
            box_min, box_max = np.array(triangle.bounds[:3]), np.array(triangle.bounds[3:])
            if ray_box_intersection(box_min, box_max):
                intersecting_triangles.append(triangle)

        return intersecting_triangles

In [2]:
tris = ObjLoader("../").load("./monkey_circle.obj")[1]

TypeError: '<=' not supported between instances of 'Vector' and 'Vector'

In [3]:
len(tris)
print(tris[0].bounds)

NameError: name 'tris' is not defined

In [4]:
rtree = RTree3D(tris)
origin = np.array([0, 0, 0])
direction = np.array([20, 0, 0])

intersected_triangles = rtree.queryRay(origin, direction)
len(intersected_triangles)

NameError: name 'tris' is not defined

In [5]:
import numpy as np
from rtree import index
from rtree.index import Rtree

class Vector:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y, self.z - other.z)

    def cross(self, other):
        return Vector(self.y * other.z - self.z * other.y,
                      self.z * other.x - self.x * other.z,
                      self.x * other.y - self.y * other.x)

class Triangle:
    def __init__(self, coords):
        self.a = coords[0]
        self.b = coords[1]
        self.c = coords[2]
        self.at = None
        self.bt = None
        self.ct = None
        self.normal = None
        self.collisions = []
        self.textureCoords = []

class TriangleRTree:
    def __init__(self, triangles):
        self.index = index.Index(interleaved=False)
        for i, t in enumerate(triangles):
            self.index.insert(i, self._triangle_bbox(t))

    def _triangle_bbox(self, triangle):
        min_x = min(triangle.a.x, triangle.b.x, triangle.c.x)
        min_y = min(triangle.a.y, triangle.b.y, triangle.c.y)
        min_z = min(triangle.a.z, triangle.b.z, triangle.c.z)
        max_x = max(triangle.a.x, triangle.b.x, triangle.c.x)
        max_y = max(triangle.a.y, triangle.b.y, triangle.c.y)
        max_z = max(triangle.a.z, triangle.b.z, triangle.c.z)
        return (min_x, min_y, min_z, max_x, max_y, max_z)

    def queryRay(self, origin, direction, triangles):
        ray_bbox = self._compute_ray_bbox(origin, direction)
        intersected_indices = list(self.index.intersection(ray_bbox, objects=False))
        intersected_triangles = [triangles[i] for i in intersected_indices]
        return intersected_triangles

    def _compute_ray_bbox(self, origin, direction, t_min=0, t_max=float('inf')):
        inv_direction = Vector(1 / direction.x, 1 / direction.y, 1 / direction.z)
        t1 = (t_min * direction + origin - Vector(0, 0, 0)) * inv_direction
        t2 = (t_max * direction + origin - Vector(0, 0, 0)) * inv_direction
        return (min(t1.x, t2.x), min(t1.y, t2.y), min(t1.z, t2.z),
                max(t1.x, t2.x), max(t1.y, t2.y), max(t1.z, t2.z))


In [6]:
tris = ObjLoader("../").load("./monkey_circle.obj")[1]
triangle_rtree = TriangleRTree(tris)

TypeError: '<=' not supported between instances of 'Vector' and 'Vector'

In [20]:
from twobounce2 import ObjLoader
from GeometricObjects import *

class Vector:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

class Triangle:
    def __init__(self, coords):
        self.a = coords[0]
        self.b = coords[1]
        self.c = coords[2]
        self.at = None
        self.bt = None
        self.ct = None
        self.normal = None
        self.collisions = []
        self.textureCoords = []

class RTreeNode:
    def __init__(self, triangles=None, children=None):
        self.triangles = triangles if triangles is not None else []
        self.children = children if children is not None else []
        self.bbox = None
        self.update_bbox()

    def update_bbox(self):
        if self.children:
            self.bbox = self.combine_bboxes([child.bbox for child in self.children])
        elif self.triangles:
            self.bbox = self.combine_bboxes([self.get_bbox(triangle) for triangle in self.triangles])

    def combine_bboxes(self, bboxes):
        if not bboxes:
            return None

        min_x = min([bbox[0] for bbox in bboxes])
        min_y = min([bbox[1] for bbox in bboxes])
        min_z = min([bbox[2] for bbox in bboxes])
        max_x = max([bbox[3] for bbox in bboxes])
        max_y = max([bbox[4] for bbox in bboxes])
        max_z = max([bbox[5] for bbox in bboxes])

        return (min_x, min_y, min_z, max_x, max_y, max_z)

    def get_bbox(self, triangle):
        coords = [triangle.a, triangle.b, triangle.c]
        min_x = min(coord.x for coord in coords)
        min_y = min(coord.y for coord in coords)
        min_z = min(coord.z for coord in coords)
        max_x = max(coord.x for coord in coords)
        max_y = max(coord.y for coord in coords)
        max_z = max(coord.z for coord in coords)

        return (min_x, min_y, min_z, max_x, max_y, max_z)

class RTREE:
    def __init__(self, max_triangles=10):
        self.root = RTreeNode()
        self.max_triangles = max_triangles

    def insert(self, triangle):
        if not self.root.bbox:
            self.root.triangles.append(triangle)
            self.root.update_bbox()
            return

        if len(self.root.triangles) < self.max_triangles:
            self.root.triangles.append(triangle)
            self.root.update_bbox()
        else:
            if not self.root.children:
                self.split_root()

            for child in self.root.children:
                if self.insert_into_node(triangle, child):
                    break

    def split_root(self):
        sorted_triangles = sorted(self.root.triangles, key=lambda t: (t.a.x + t.b.x + t.c.x) / 3)
        half = len(sorted_triangles) // 2
        self.root.children = [
            RTreeNode(triangles=sorted_triangles[:half]),
            RTreeNode(triangles=sorted_triangles[half:]),
        ]
        self.root.triangles = []
        self.root.update_bbox()

    def insert_into_node(self, triangle, node):
        if not node.bbox:
            node.triangles.append(triangle)
            node.update_bbox()
            return True

        if len(node.triangles) < self.max_triangles:
            node.triangles.append(triangle)
            node.update_bbox()
            return True
        else:
            if not node.children:
                self.split_node(node)

            for child in node.children:
                if self.insert_into_node(triangle, child):
                    return True

        return False

    def split_node(self, node):
        sorted_triangles = sorted(node.triangles, key=lambda t: (t.a.x + t.b.x + t.c.x) / 3)
        half = len(sorted_triangles) // 2
        node.children = [
            RTreeNode(triangles=sorted_triangles[:half]),
            RTreeNode(triangles=sorted_triangles[half:]),
        ]
        node.triangles = []
        node.update_bbox()

    def query_ray(self, origin, direction):
        intersected_triangles = []
        self._query_ray_recursive(origin, direction, self.root, intersected_triangles)
        return intersected_triangles

    def _query_ray_recursive(self, origin, direction, node, intersected_triangles):
        if not node.bbox or self.ray_intersects_bbox(origin, direction, node.bbox):
            if node.children:
                for child in node.children:
                    self._query_ray_recursive(origin, direction, child, intersected_triangles)
            else:
                intersected_triangles.extend(node.triangles)

    def ray_intersects_bbox(self, origin, direction, bbox):
        min_x, min_y, min_z, max_x, max_y, max_z = bbox
        tmin = (min_x - origin.x) / direction.x
        tmax = (max_x - origin.x) / direction.x

        if tmin > tmax:
            tmin, tmax = tmax, tmin

        tymin = (min_y - origin.y) / direction.y
        tymax = (max_y - origin.y) / direction.y

        if tymin > tymax:
            tymin, tymax = tymax, tymin

        if (tmin > tymax) or (tymin > tmax):
            return False

        tmin = max(tmin, tymin)
        tmax = min(tmax, tymax)

        tzmin = (min_z - origin.z) / direction.z
        tzmax = (max_z - origin.z) / direction.z

        if tzmin > tzmax:
            tzmin, tzmax = tzmax, tzmin

        if (tmin > tzmax) or (tzmin > tmax):
            return False

        return True



In [23]:
tris = ObjLoader("../").load("./monkey_circle.obj")[1]
rtree = RTREE()
for tri in tris:
    rtree.insert(tri)
rtree.root.

[Tri[<-2.3,  0.1,  0.2>, <-2.3,  0.0,  0.4>, <-2.4, -0.0,  0.4>],
 Tri[<-2.2,  0.2, -0.1>, <-2.3,  0.1, -0.1>, <-2.2,  0.1, -0.2>],
 Tri[<-2.2,  0.2,  0.2>, <-2.3,  0.1,  0.2>, <-2.3,  0.2,  0.1>],
 Tri[<-2.1,  0.2, -0.2>, <-2.2,  0.2, -0.1>, <-2.2,  0.1, -0.2>],
 Tri[<-2.2,  0.2,  0.2>, <-2.2,  0.2,  0.3>, <-2.2,  0.1,  0.2>],
 Tri[<-2.1,  0.2, -0.2>, <-2.2,  0.4, -0.2>, <-2.2,  0.2, -0.1>],
 Tri[<-2.2,  0.4,  0.2>, <-2.2,  0.2,  0.3>, <-2.2,  0.2,  0.2>],
 Tri[<-2.2,  0.4, -0.2>, <-2.3,  0.2, -0.0>, <-2.2,  0.2, -0.1>],
 Tri[<-2.2,  0.4,  0.2>, <-2.3,  0.2,  0.1>, <-2.3,  0.4,  0.2>],
 Tri[<-2.2,  0.5, -0.3>, <-2.3,  0.4, -0.1>, <-2.2,  0.4, -0.2>]]

In [13]:
len(tris)

7744

In [73]:
import math

class BoundingBox:
    def __init__(self, min_point, max_point):
        self.min_point = min_point
        self.max_point = max_point

    def intersects(self, origin, direction):
        t1 = (self.min_point[0] - origin[0]) / direction[0]
        t2 = (self.max_point[0] - origin[0]) / direction[0]
        t3 = (self.min_point[1] - origin[1]) / direction[1]
        t4 = (self.max_point[1] - origin[1]) / direction[1]
        t5 = (self.min_point[2] - origin[2]) / direction[2]
        t6 = (self.max_point[2] - origin[2]) / direction[2]

        tmin = max(min(t1, t2), min(t3, t4), min(t5, t6))
        tmax = min(max(t1, t2), max(t3, t4), max(t5, t6))

        return tmax > max(tmin, 0)

class RTreeNode:
    def __init__(self, bounding_box, triangles=None, children=None):
        self.bounding_box = bounding_box
        self.triangles = triangles if triangles is not None else []
        self.children = children if children is not None else []
class RTree:
    def __init__(self, triangles, max_triangles_per_leaf=8):
        self.root = self.build_tree(triangles, max_triangles_per_leaf)

    def build_tree(self, triangles, max_triangles_per_leaf):
        if len(triangles) <= max_triangles_per_leaf:
            min_point, max_point = self.compute_bounds(triangles)
            bounding_box = BoundingBox(min_point, max_point)
            return RTreeNode(bounding_box, triangles=triangles)

        # Choose an axis to split along
        axis = self.choose_split_axis(triangles)

        # Sort triangles along the chosen axis
        triangles.sort(key=lambda t: t.a[axis])

        # Split triangles into two halves
        mid = len(triangles) // 2
        left_triangles = triangles[:mid]
        right_triangles = triangles[mid:]

        left_child = self.build_tree(left_triangles, max_triangles_per_leaf)
        right_child = self.build_tree(right_triangles, max_triangles_per_leaf)

        min_point = [
            min(left_child.bounding_box.min_point[i], right_child.bounding_box.min_point[i]) for i in range(3)
        ]
        max_point = [
            max(left_child.bounding_box.max_point[i], right_child.bounding_box.max_point[i]) for i in range(3)
        ]
        bounding_box = BoundingBox(min_point, max_point)

        return RTreeNode(bounding_box, children=[left_child, right_child])

    def compute_bounds(self, triangles):
        min_point = [math.inf] * 3
        max_point = [-math.inf] * 3

        for triangle in triangles:
            for vertex in [triangle.a, triangle.b, triangle.c]:
                for i in range(3):
                    min_point[i] = min(min_point[i], vertex[i])
                    max_point[i] = max(max_point[i], vertex[i])

        return min_point, max_point

    def choose_split_axis(self, triangles):
        extents = []
        for axis in range(3):
            min_coord, max_coord = zip(*[(t.a[axis], t.c[axis]) for t in triangles])
            extents.append(max(max_coord) - min(min_coord))
        return extents.index(max(extents))

    def query_ray(self, origin, direction):
        return self._query_ray_recursive(self.root, origin, direction)

    def _query_ray_recursive(self, node, origin, direction):
        if not node.bounding_box.intersects(origin, direction):
            return []

        if node.children:
            result = []
            for child in node.children:
                result.extend(self._query_ray_recursive(child, origin, direction))
            return result
        else:
            return node.triangles

triangles = []  # List of Triangle objects
rtree = RTree(triangles)
origin = [0, 0, 0]
direction = [1, 1, 1]
intersected_triangles = rtree.query_ray(origin, direction)
intersected_triangles



TypeError: 'Vector' object is not subscriptable

In [35]:
import random
def random_vector():
    return [random.uniform(-100, 100) for _ in range(3)]

def random_triangle():
    return Triangle([random_vector() for _ in range(3)])

# Generate a list of random triangles
num_triangles = 100
triangles = [random_triangle() for _ in range(num_triangles)]
rtree = RTree(triangles)
origin = [0, 0, 0]
direction = [100, 0.1, 0.1]
intersected_triangles = rtree.query_ray(origin, direction)
len(intersected_triangles)

100

In [53]:
tree = rtree.root
while(len(tree.children)):

    print(tree.children)
    tree = tree.children[0]

[<__main__.RTreeNode object at 0x1566711d0>, <__main__.RTreeNode object at 0x1566719d0>]
[<__main__.RTreeNode object at 0x15666bc50>, <__main__.RTreeNode object at 0x15666a3d0>]
[<__main__.RTreeNode object at 0x15666b7d0>, <__main__.RTreeNode object at 0x15666be10>]
[<__main__.RTreeNode object at 0x15666b290>, <__main__.RTreeNode object at 0x15666b310>]


In [61]:
tree.bounding_box.intersects((0.01, 0.01, 0.01), (1, 0.01, 0.01))

True

In [59]:
tree.bounding_box.min_point

[-99.78857440376355, -85.51171711371464, -68.73622275382793]

In [62]:
tree.bounding_box.max_point

[99.54372129396623, 87.59496997493989, 86.57704565192074]

In [63]:
rtree.root.bounding_box.max_point

[99.70778236779182, 99.96331616918184, 99.30647976197272]

In [78]:
tree = rtree.root
def printnode(tree):
    return f"{tree.bounding_box.min_point}, {tree.bounding_box.max_point}"
def print_r_tree(tree, level):
    print("\t"*level + printnode(tree))
    if(tree.children):
        print_r_tree(tree.children[0], level+1)
        print_r_tree(tree.children[1], level+1)
print_r_tree(tree, 0)

[-10, -10, -10], [10, 10, 10]
	[-10, -10, -10], [0, 10, 10]
		[-10, 0, -10], [0, 10, 10]
			[-10, 0, 0], [0, 10, 10]
			[-10, 0, -10], [0, 10, 0]
		[-10, -10, -10], [0, 0, 10]
			[-10, -10, 0], [0, 0, 10]
			[-10, -10, -10], [0, 0, 0]
	[0, -10, -10], [10, 10, 10]
		[0, 0, -10], [10, 10, 10]
			[0, 0, 0], [10, 10, 10]
			[0, 0, -10], [10, 10, 0]
		[0, -10, -10], [10, 0, 10]
			[0, -10, 0], [10, 0, 10]
			[0, -10, -10], [10, 0, 0]


In [72]:
import random
def random_vector():
    return [random.uniform(-100, 100) for _ in range(3)]

def random_triangle():
    vec = random_vector()
    print(vec)
    return Triangle([vec for _ in range(3)])

# Generate a list of random triangles
num_triangles = 3
triangles = [random_triangle() for _ in range(num_triangles)]

def compute_bounds(triangles):
        min_point = [math.inf] * 3
        max_point = [-math.inf] * 3

        for triangle in triangles:
            for vertex in [triangle.a, triangle.b, triangle.c]:
                for i in range(3):
                    min_point[i] = min(min_point[i], vertex[i])
                    max_point[i] = max(max_point[i], vertex[i])

        return min_point, max_point

compute_bounds(triangles)

[-33.356823213659226, 43.78979123383891, 66.0943888773289]
[-38.328655211008275, -52.138041232992485, -36.69493469567335]
[-76.85302275092474, -98.48395154020784, 63.447123056270414]


([-76.85302275092474, -98.48395154020784, -36.69493469567335],
 [-33.356823213659226, 43.78979123383891, 66.0943888773289])

In [70]:
]

In [76]:
def create_triangle_in_octant(octant):
    base_coords = [
        (10, 0, 0),
        (0, 10, 0),
        (0, 0, 10),
    ]
    
    triangle_coords = [
        [(coord if octant[i] else -coord) for i, coord in enumerate(vertex)] for vertex in base_coords
    ]
    
    return Triangle(triangle_coords)

# Generate a list of triangles, one in each octant
triangles = [create_triangle_in_octant(octant) for octant in [
    (True, True, True),
    (True, True, False),
    (True, False, True),
    (True, False, False),
    (False, True, True),
    (False, True, False),
    (False, False, True),
    (False, False, False),
]]

# Build the R-Tree
rtree = RTree(triangles, 1)

# Define a ray (origin and direction)
origin = [0, 0, 0]
direction = [1, 1, 1]

# Query the R-Tree with the ray
intersected_triangles = rtree.query_ray(origin, direction)

# Print the result
print("Ray origin:", origin)
print("Ray direction:", direction)
print("Intersected triangles:")
for t in intersected_triangles:
    print("Triangle:", t.a, t.b, t.c)


Ray origin: [0, 0, 0]
Ray direction: [1, 1, 1]
Intersected triangles:
Triangle: [10, 0, 0] [0, 10, 0] [0, 0, 10]
